[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/python35/IINTS-SDK/blob/main/examples/notebooks/02_Safety_and_Supervisor.ipynb)


In [1]:
import os
import sys
from pathlib import Path
import subprocess

def _ensure_repo():
    workspace = os.environ.get('GITHUB_WORKSPACE')
    if workspace:
        repo_root = Path(workspace).resolve()
        src_path = repo_root / 'src'
        if src_path.exists():
            sys.path.insert(0, str(src_path))
            return repo_root
    repo_root = Path().resolve()
    src_path = repo_root / 'src'
    if src_path.exists():
        sys.path.insert(0, str(src_path))
        return repo_root
    repo_dir = Path('IINTS-SDK')
    if not repo_dir.exists():
        subprocess.check_call(['git', 'clone', 'https://github.com/python35/IINTS-SDK.git'])
    repo_root = repo_dir.resolve()
    os.chdir(repo_root)
    src_path = repo_root / 'src'
    sys.path.insert(0, str(src_path))
    return repo_root

_ensure_repo()


PosixPath('/home/runner/work/IINTS-SDK/IINTS-SDK')

# Supervisor and Safety Rules

Inspect the safety layer and see how decisions are modified.


In [2]:
from iints.core.supervisor import IndependentSupervisor

supervisor = IndependentSupervisor(max_insulin_per_bolus=2.0, max_60min=3.0)

decision = supervisor.evaluate_safety(
    current_glucose=75,
    proposed_insulin=3.0,
    current_time=0.0,
    current_iob=1.5,
)

decision


{'approved_insulin': 2.0,
 'safety_level': <SafetyLevel.WARNING: 'warning'>,
 'actions_taken': ['LIMIT: Bolus capped at 2.0U'],
 'original_insulin': 3.0,
 'insulin_reduction': 1.0,
 'emergency_mode': False,
 'safety_decision': SafetyDecision(original_dose=3.0, final_dose=2.0, reason='LIMIT: Bolus capped at 2.0U', triggered=True),
 'safety_reason': 'LIMIT: Bolus capped at 2.0U',
 'safety_triggered': True}

## Safety report after a short run


In [3]:
import json
from iints.presets import load_presets

import iints
from iints.core.algorithms.correction_bolus import CorrectionBolus

presets = load_presets()
preset = next(p for p in presets if p["name"] == "pizza_paradox")

outputs = iints.run_simulation(
    algorithm=CorrectionBolus,
    scenario=preset["scenario"],
    patient_config=preset["patient_config"],
    duration_minutes=240,
    time_step=preset["time_step_minutes"],
    seed=7,
    output_dir="results/safety_demo",
)

outputs["safety_report"]


Simulation terminated early: Critical failure: glucose < 40.0 mg/dL for 30 minutes.


Simulation terminated early: Critical failure: glucose < 40.0 mg/dL for 30 minutes.


Simulation terminated early: Critical failure: glucose < 40.0 mg/dL for 30 minutes.


{'total_violations': 13,
 'violation_breakdown': {'safe': 3,
  'warning': 0,
  'critical': 2,
  'emergency': 8},
 'bolus_interventions_count': 13,
 'emergency_mode_active': True,
 'current_iob': 0.6005555555555555,
 'recent_violations': [{'level': 'emergency',
   'message': 'HYPO_CUTOFF: Glucose below safety cutoff; EMERGENCY_STOP: Severe hypoglycemia detected',
   'glucose': 28.62991551780664,
   'time': 45.0,
   'original_proposed_insulin': 0.06666666666666667,
   'approved_insulin': 0},
  {'level': 'emergency',
   'message': 'HYPO_CUTOFF: Glucose below safety cutoff; EMERGENCY_STOP: Severe hypoglycemia detected',
   'glucose': 23.483576338283793,
   'time': 50.0,
   'original_proposed_insulin': 0.06666666666666667,
   'approved_insulin': 0},
  {'level': 'emergency',
   'message': 'HYPO_CUTOFF: Glucose below safety cutoff; EMERGENCY_STOP: Severe hypoglycemia detected',
   'glucose': 20.0,
   'time': 55.0,
   'original_proposed_insulin': 0.06666666666666667,
   'approved_insulin': 0},